In [ ]:
# notebooks/initial_testing.ipynb

# 1. Cài đặt và Thiết lập Môi trường
# !pip install -r ../requirements.txt 
# import os
# os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"

# 2. Import các Module cần thiết
import sys
import os

# Thêm đường dẫn để có thể import các module src
sys.path.append(os.path.join(os.getcwd(), '..'))

from scripts.run_indexing import run_indexing
from src.rag_pipeline import run_rag_query
from src.core.vector_store import initialize_vector_store

# 3. CHẠY INDEXING (Đảm bảo có file trong data/raw trước)
# Đây là bước quan trọng nhất, chỉ chạy khi có dữ liệu mới hoặc cập nhật
# run_indexing() 

# Sau khi chạy indexing, bạn có thể tải Vector Store mà không cần tạo lại
vectordb = initialize_vector_store()
print(f"Tổng số chunks trong DB: {vectordb._collection.count()}")

# 4. KIỂM TRA CHỨC NĂNG RETRIEVAL (Truy xuất)
# Kiểm tra xem Retriever có tìm đúng đoạn văn bản không
test_query = "Số lượng tình nguyện viên Ban Giao lưu được yêu cầu là bao nhiêu?"
retrieved_docs = vectordb.as_retriever(search_kwargs={"k": 2}).invoke(test_query)

print("\n--- Kết quả Truy xuất (Retrieval Test) ---")
for i, doc in enumerate(retrieved_docs):
    print(f"Chunk {i+1} (Source: {doc.metadata.get('source')}): {doc.page_content[:300]}...")

# 5. KIỂM TRA TOÀN BỘ RAG PIPELINE
# Kiểm tra xem LLM có thể sinh đáp án dựa trên context được cung cấp không
rag_result = run_rag_query(test_query)

print("\n--- Kết quả RAG (Generation Test) ---")
print(f"Câu hỏi: {test_query}")
print(f"Đáp án: {rag_result.get('answer')}")

# Hiển thị các nguồn đã được truy xuất
print("\nNGUỒN TRUY XUẤT:")
for doc in rag_result.get("context", []):
    print(f"- {doc.metadata.get('source')}")